# Step 3: Agents in a Workflow with Streaming

## Overview

This notebook demonstrates **streaming workflow execution**, allowing you to observe events as they occur in real-time. We'll build a two-agent workflow with:
1. **Writer Agent** - Generates content
2. **Reviewer Agent** - Reviews and finalizes the result

### Key Concepts:

- **Streaming Execution**: Using `workflow.run_stream()` to observe events in real-time
- **Custom Executor Classes**: Creating reusable executor components with embedded agents
- **Event Types**: Understanding different workflow events (status, output, errors)
- **Event Origin**: Distinguishing runner-generated vs executor-generated events
- **Typed Contexts**: Using `WorkflowContext[T_Out, T_W_Out]` for type safety

### Prerequisites:

- ✅ Azure OpenAI configured with required environment variables
- ✅ Azure CLI authentication (`az login` completed)
- ✅ Completion of Step 1 and Step 2

## Import Required Libraries

In [ ]:
import asyncio

from agent_framework import (
    ChatAgent,
    ChatMessage,
    Executor,
    ExecutorFailedEvent,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowFailedEvent,
    WorkflowRunState,
    WorkflowStatusEvent,
    handler,
)
from agent_framework._workflows._events import WorkflowOutputEvent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from typing_extensions import Never
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables from .env file
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv('../../.env')

## Create Azure OpenAI Chat Client

We'll use **Azure OpenAI** with `AzureCliCredential` for authentication.

### Environment Variables Required:
- **AZURE_OPENAI_ENDPOINT**: Your Azure OpenAI endpoint URL
- **AZURE_OPENAI_CHAT_DEPLOYMENT_NAME**: Your model deployment name (e.g., "gpt-4o")

These should be set in the `.env` file located at `python/samples/getting_started/.env`

In [ ]:
# Verify environment variables are loaded
print("🔍 Checking Azure OpenAI environment variables...")
print(f"AZURE_OPENAI_ENDPOINT: {os.getenv('AZURE_OPENAI_ENDPOINT', 'Not set')}")
print(f"AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: {os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'Not set')}")

# Create the Azure OpenAI chat client
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

if not endpoint:
    raise ValueError("❌ Azure OpenAI endpoint not found. Please set AZURE_OPENAI_ENDPOINT in your .env file")

if not deployment_name:
    raise ValueError("❌ Azure OpenAI deployment name not found. Please set AZURE_OPENAI_CHAT_DEPLOYMENT_NAME in your .env file")

print(f"🔧 Using Azure OpenAI Endpoint: {endpoint}")
print(f"🔧 Using Deployment: {deployment_name}")

# Create the Azure OpenAI chat client
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)

print("✅ Azure OpenAI Chat Client created successfully!")

## Define the Writer Executor

### Custom Executor Pattern

This class demonstrates:
- Attaching a `ChatAgent` to an `Executor` for workflow participation
- Using `@handler` method with typed input and output
- Forwarding typed output via `ctx.send_message()`

### Handler Contract:
- **Input**: `ChatMessage` (inbound user message)
- **Context**: `WorkflowContext[list[ChatMessage]]` (expects list of messages downstream)

### Pattern:
1. Seed conversation with inbound message
2. Run attached agent to produce assistant messages
3. Forward cumulative messages to next executor

In [ ]:
class Writer(Executor):
    """Custom executor that owns a domain-specific agent for content generation."""

    agent: ChatAgent

    def __init__(self, chat_client: AzureOpenAIChatClient, id: str = "writer"):
        # Create a domain-specific agent using your configured AzureOpenAIChatClient
        self.agent = chat_client.create_agent(
            instructions=(
                "You are an excellent content writer. You create new content and edit contents based on the feedback."
            ),
        )
        # Associate this agent with the executor node
        super().__init__(id=id)

    @handler
    async def handle(self, message: ChatMessage, ctx: WorkflowContext[list[ChatMessage]]) -> None:
        """Generate content and forward the updated conversation.

        Contract for this handler:
        - message is the inbound user ChatMessage
        - ctx is a WorkflowContext that expects a list[ChatMessage] to be sent downstream
        """
        # Start the conversation with the incoming user message
        messages: list[ChatMessage] = [message]
        
        # Run the agent and extend the conversation with the agent's messages
        response = await self.agent.run(messages)
        messages.extend(response.messages)
        
        # Forward the accumulated messages to the next executor in the workflow
        await ctx.send_message(messages)

print("✅ Writer Executor class defined")

## Define the Reviewer Executor

### Terminal Node Pattern

This executor demonstrates:
- Consuming the full conversation transcript
- Using `WorkflowContext[Never, str]` for terminal nodes
  - `Never` = does not send messages to downstream nodes
  - `str` = yields string as final workflow output
- Completing the workflow with `ctx.yield_output()`

### Handler Contract:
- **Input**: `list[ChatMessage]` (full conversation history)
- **Context**: `WorkflowContext[Never, str]` (terminal node yielding string output)

In [ ]:
class Reviewer(Executor):
    """Custom executor that owns a review agent and completes the workflow."""

    agent: ChatAgent

    def __init__(self, chat_client: AzureOpenAIChatClient, id: str = "reviewer"):
        # Create a domain-specific agent that evaluates and refines content
        self.agent = chat_client.create_agent(
            instructions=(
                "You are an excellent content reviewer. You review the content and provide feedback to the writer."
            ),
        )
        super().__init__(id=id)

    @handler
    async def handle(self, messages: list[ChatMessage], ctx: WorkflowContext[Never, str]) -> None:
        """Review the full conversation transcript and yield the final output.

        This node consumes all messages so far. It uses its agent to produce the final text,
        then yields the output. The workflow completes when it becomes idle.
        """
        response = await self.agent.run(messages)
        await ctx.yield_output(response.text)

print("✅ Reviewer Executor class defined")

## Create Executor Instances

In [ ]:
# Instantiate the two agent-backed executors
writer = Writer(chat_client)
reviewer = Reviewer(chat_client)

print("✅ Writer and Reviewer instances created")

## Build the Workflow

### Workflow Structure:

```
Writer → Reviewer
```

In [ ]:
# Build the workflow using the fluent builder
workflow = (
    WorkflowBuilder()
    .set_start_executor(writer)
    .add_edge(writer, reviewer)
    .build()
)

print("✅ Workflow built successfully!")
print("   Writer → Reviewer")

## Run the Workflow with Streaming

### Streaming Events

Using `workflow.run_stream()` provides real-time visibility into:
- **WorkflowStatusEvent**: State changes (IN_PROGRESS, IDLE, etc.)
- **WorkflowOutputEvent**: Final outputs from terminal nodes
- **ExecutorInvokeEvent**: When executors are invoked
- **ExecutorCompletedEvent**: When executors complete
- **ExecutorFailedEvent**: Executor-level errors
- **WorkflowFailedEvent**: Workflow-level failures

### Event Origin

Each event has an `origin` field:
- **RUNNER**: Lifecycle events (state changes, invocations)
- **EXECUTOR**: Data-plane events (outputs, errors)

### Workflow States

- `IN_PROGRESS` - Workflow is actively processing
- `IN_PROGRESS_PENDING_REQUESTS` - Processing with external requests in flight
- `IDLE` - No active work, workflow complete or waiting
- `IDLE_WITH_PENDING_REQUESTS` - Waiting for user input or external data

In [ ]:
# User message
user_message = "Create a slogan for a new electric SUV that is affordable and fun to drive."

print(f"\n📝 User Request: {user_message}\n")
print("=" * 70)
print("\n🔄 Streaming Events:\n")

# Run the workflow with streaming to observe events as they occur
async for event in workflow.run_stream(
    ChatMessage(role="user", text=user_message)
):
    if isinstance(event, WorkflowStatusEvent):
        prefix = f"State ({event.origin.value}): "
        if event.state == WorkflowRunState.IN_PROGRESS:
            print(prefix + "IN_PROGRESS")
        elif event.state == WorkflowRunState.IN_PROGRESS_PENDING_REQUESTS:
            print(prefix + "IN_PROGRESS_PENDING_REQUESTS (requests in flight)")
        elif event.state == WorkflowRunState.IDLE:
            print(prefix + "IDLE (no active work)")
        elif event.state == WorkflowRunState.IDLE_WITH_PENDING_REQUESTS:
            print(prefix + "IDLE_WITH_PENDING_REQUESTS (prompt user or UI now)")
        else:
            print(prefix + str(event.state))
    
    elif isinstance(event, WorkflowOutputEvent):
        print(f"\n📤 Workflow output ({event.origin.value}): {event.data}")
    
    elif isinstance(event, ExecutorFailedEvent):
        print(
            f"❌ Executor failed ({event.origin.value}): "
            f"{event.executor_id} {event.details.error_type}: {event.details.message}"
        )
    
    elif isinstance(event, WorkflowFailedEvent):
        details = event.details
        print(f"❌ Workflow failed ({event.origin.value}): {details.error_type}: {details.message}")
    
    else:
        print(f"{event.__class__.__name__} ({event.origin.value}): {event}")

print("\n" + "=" * 70)
print("\n✅ Workflow execution completed!")

## Expected Output

### Sample Streaming Events:

```
State (RUNNER): IN_PROGRESS
ExecutorInvokeEvent (RUNNER): ExecutorInvokeEvent(executor_id=writer)
ExecutorCompletedEvent (RUNNER): ExecutorCompletedEvent(executor_id=writer)
ExecutorInvokeEvent (RUNNER): ExecutorInvokeEvent(executor_id=reviewer)
📤 Workflow output (EXECUTOR): Drive the Future. Affordable Adventure, Electrified.
ExecutorCompletedEvent (RUNNER): ExecutorCompletedEvent(executor_id=reviewer)
State (RUNNER): IDLE
```

### Event Flow Explanation:

1. **IN_PROGRESS** - Workflow starts processing
2. **ExecutorInvokeEvent (writer)** - Writer agent is invoked
3. **ExecutorCompletedEvent (writer)** - Writer completes and forwards messages
4. **ExecutorInvokeEvent (reviewer)** - Reviewer agent is invoked
5. **WorkflowOutputEvent** - Reviewer yields final output
6. **ExecutorCompletedEvent (reviewer)** - Reviewer completes
7. **IDLE** - Workflow becomes idle and completes

## Key Takeaways

### Streaming vs Non-Streaming

| Feature | `run()` (Non-Streaming) | `run_stream()` (Streaming) |
|---------|------------------------|---------------------------|
| **Execution** | Waits for completion | Real-time events |
| **Return Type** | Event collection | Async iterator |
| **User Experience** | Batch processing | Real-time feedback |
| **Use Case** | Simple workflows | Interactive applications |
| **Covered In** | Step 2 | Step 3 (this notebook) |

### Custom Executor Pattern

✅ **When to Use Custom Executors:**
- Need to manage agent state
- Require lifecycle hooks (initialization, cleanup)
- Building reusable workflow components
- Domain-specific agent configurations

✅ **Handler Signature Pattern:**
```python
@handler
async def handle(self, input: InputType, ctx: WorkflowContext[OutputType]) -> None:
    # Process input
    # Send to downstream nodes: await ctx.send_message(output)
    # Or yield final output: await ctx.yield_output(result)
```

### Event Types and Origin

**RUNNER Events (Lifecycle):**
- `WorkflowStatusEvent` - State transitions
- `ExecutorInvokeEvent` - Executor invocations
- `ExecutorCompletedEvent` - Executor completions

**EXECUTOR Events (Data-Plane):**
- `WorkflowOutputEvent` - Final outputs
- `ExecutorFailedEvent` - Executor errors

### Message Flow Pattern

```
User Input (ChatMessage)
    ↓
Writer Handler
    ├─ Creates conversation: [user_message]
    ├─ Runs writer agent → adds assistant messages
    └─ Forwards: ctx.send_message(all_messages)
    ↓
Reviewer Handler
    ├─ Receives: list[ChatMessage]
    ├─ Runs reviewer agent on full conversation
    └─ Yields: ctx.yield_output(final_text)
    ↓
Workflow Output
```

### Next Steps

Explore advanced workflow patterns:
- **Control Flow**: Conditional edges, loops, switch-case
- **Parallelism**: Fan-out/fan-in, concurrent execution
- **Orchestration**: Multi-agent coordination
- **Human-in-the-Loop**: Interactive workflows with user approval